## Face Verification using an AutoEncoder

In [ ]:
import numpy as np ## for numerical calculations of arrays
import pandas as pd ## for reading csv file and wroking with dataframe operations
from PIL import Image ## for image processing and output
import cv2 ## for image processing
import numpy
import os ## for reading images from image folder
import matplotlib.pyplot as plt
import random

In [ ]:
import tensorflow as tf
from keras.backend import epsilon
from tensorflow.keras.layers import InputLayer,Input,Dense, Conv2D, MaxPooling2D, UpSampling2D,Conv2DTranspose, Flatten, Reshape
from tensorflow.keras.models import Model,Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
random.seed(42)

## Face detection and cropping

In [ ]:
# Load OpenCV's pre-trained face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

## Reading the Dataset
We're reading the folders and splitting them into train and test set for training purposes.

In [ ]:
# Set paths
dataset_folder = '/kaggle/input/iiitb-faces/IIITB-FACES'
test_filepaths = [] # Contains the absolute paths of test images
train_filepaths = []# Contains the absolute paths of train images

# Loop through each person's folder
for person_folder in os.listdir(dataset_folder):
    person_path = os.path.join(dataset_folder, person_folder)
    person_images = [os.path.join(person_path, image_file) for image_file in os.listdir(person_path)]

    random.shuffle(person_images)
    # Calculate split point based on 80-20 ratio
    split_index = int(0.8 * len(person_images))
    
    # Split images into train and test
    train_filepaths.append(person_images[:split_index])
    test_filepaths.append(person_images[split_index:])

print("Total images:", len(train_filepaths)+len(test_filepaths))
print("Total train images:", len(train_filepaths))
print("Total test images:", len(test_filepaths))

In [ ]:
total_images = len(train_filepaths)+len(test_filepaths)
total_train_images = len(train_filepaths)
total_test_images = len(test_filepaths)
total_classes = 49

In [ ]:
transformed_train_filepaths = []
for filename in train_filepaths:
    temp = []
    for imagename in filename:
        img = cv2.imread(imagename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        face_image = img
        if(len(faces) > 0):
            xf, yf, wf, hf = faces[0]
            face_image = img[yf:yf+hf, xf:xf+wf]
        face_image = cv2.resize(face_image, (512, 512))
        

        temp.append(face_image)
    transformed_train_filepaths.append(temp)

In [ ]:
sum = 0
lis = []
frequency = {}
for i in range(0, len(transformed_train_filepaths)):
    sum += ((len(transformed_train_filepaths[i]))*(len(transformed_train_filepaths[i]) - 1))/2
    lis.append(len(transformed_train_filepaths[i]))

for item in lis:
   # checking the element in dictionary
   if item in frequency:
      # incrementing the count
      frequency[item] += 1
   else:
      # initializing the count
      frequency[item] = 1
tot_val = 0
for key in frequency:
    tot_val += key*frequency[key]
final_freq = {}
for key in frequency:
    final_freq[key] = key*frequency[key]/tot_val


# printing the frequency
print(final_freq)

## Creating pairs of images from the training dataset

In [ ]:
random.seed(42)
newX1 = []
newX2 = []
newY = []
for i in range(len(transformed_train_filepaths)):

    for j in range(0, len(transformed_train_filepaths[i])):
        for k in range(0, j):
            newX1.append(transformed_train_filepaths[i][k])
            newX2.append(transformed_train_filepaths[i][j])
            newY.append(0)

    
    for u in range(0, len(transformed_train_filepaths[i])):
        step = 1
        step = round((final_freq[len(transformed_train_filepaths[i])]*4096)/frequency[len(transformed_train_filepaths[i])])
        no_of_iter = round(step/len(transformed_train_filepaths[i]))
        for l in range(0, no_of_iter):
            numbers = list(range(0, i)) + list(range(i+1, 49))
            r = random.choice(numbers)
            g = random.randint(0, len(transformed_train_filepaths[r]) - 1)
            newX1.append(transformed_train_filepaths[i][u])
            newX2.append(transformed_train_filepaths[r][g])
            newY.append(1)
for i in range(0, 214):
        
    numbers = list(range(0,i%49)) + list(range(i%49 + 1,49))
    r = random.choice(numbers)
    uu = random.randint(0,len(transformed_train_filepaths[i%49])-1)
    g = random.randint(0,len(transformed_train_filepaths[r])-1)
    newX1.append(transformed_train_filepaths[i%49][uu])
    newX2.append(transformed_train_filepaths[r][g])
    newY.append(1)
print(len(newY))

In [ ]:
c = list(zip(newX1, newX2, newY))

random.shuffle(c)

a, b, y = zip(*c)

In [ ]:
train_images = []
for i in range(0, len(transformed_train_filepaths)):
    train_images.extend(transformed_train_filepaths[i])
print(len(train_images))

In [ ]:
train_images=np.array(train_images)
train_images=train_images.astype('float32')/np.max(train_images)


## Define Auto-Encoder Architecture


In [ ]:
def autoencoder(img_size, code_size):
    encoder = Sequential()
    encoder.add(InputLayer(img_size))
    encoder.add(Conv2D(filters=4, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(MaxPooling2D((2, 2), padding='valid'))
    encoder.add(Conv2D(filters=16, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(Conv2D(filters=16, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(MaxPooling2D((2, 2), padding='valid'))
    encoder.add(Conv2D(filters=32, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(Conv2D(filters=64, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(Conv2D(filters=128, kernel_size=(3, 3),strides=(2,2), activation='relu', padding='same'))
    encoder.add(MaxPooling2D((2, 2), padding='valid'))
    encoder.add(Flatten())
    encoder.add(Dense(units=code_size))
    
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(units=1 * 1 * 1024))
    decoder.add(Reshape((1, 1, 1024)))
    decoder.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(UpSampling2D((2, 2)))
    decoder.add(Conv2DTranspose(16, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(Conv2DTranspose(16, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(UpSampling2D((2, 2)))
    decoder.add(Conv2DTranspose(4, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    decoder.add(Conv2DTranspose(3, (3, 3), strides=(2, 2), activation='sigmoid', padding='same'))
    
    return encoder, decoder

In [ ]:
encoder, decoder = autoencoder((512, 512, 3), 512)
encoder.summary()
decoder.summary()

In [ ]:
inp = Input((512, 512, 3))
autoencoder = Model(inputs=inp, outputs=decoder(encoder(inp)))
autoencoder.compile(optimizer='adam', loss='mse')

### Fitting the AutoEncoder

In [ ]:
epochs = 150
batch_size = 16

history = autoencoder.fit(train_images, train_images, epochs=epochs, batch_size=batch_size)

## Extracting features using the encoder block of the AutoEncoder

In [ ]:
output1 = []
output2 = []
for i in range(1, len(a)//256 + 1):
    a1 = a[(i-1)*256 : i*256]
    b1 = b[(i-1)*256 : i*256]
    output1.extend(encoder.predict(np.array(a1)/255))
    output2.extend(encoder.predict(np.array(b1)/255))

In [ ]:
indexes1=[i for i,x in enumerate(y) if x == 1]
indexes0=[i for i,x in enumerate(y) if x == 0]

## Features Subtraction

- Getting absolute value of the difference between feature vectors of two images
- Plotting graphs for positive and negative image pairs

In [ ]:
arr=[]
su=[]
for s in range(len(output1)):
    oo = np.abs(np.subtract(np.array(output1[s]),np.array(output2[s])))
    arr.append(oo)
    su.append(oo.sum())
    
a = np.array(su)
su1=list(a[indexes1])
su0=list(a[indexes0])

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Sum differences")
axs[0].plot(list(range(4096)),su1, list(range(4096)),su0)
axs[0].legend(["different people", "same person"])
#axs[0].title("Euclidean distance")
axs[1].plot(list(range(8192)),su)
axs[1].legend(["overall variation"])

## Using a simple neural network for classifying the image pairs as those of the same person or those of different people¶

In [ ]:
import tensorflow.keras.backend as K
import tensorflow


def distance(vecs):
    x, y = vecs
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    
    return K.abs(x-y)


featuresA=Input(512, )
featuresB=Input(512, )
distance= Lambda(distance)([featuresA,featuresB])

x= Dense(96, activation="relu")(distance)
x= Dropout(0.3)(x)
x= Dense(64)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs=[featuresA, featuresB],outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()


### Fitting the classifier and checking the validation accuracy

In [ ]:
history=model.fit([np.array(output1)[:6144], np.array(output2)[:6144]],np.array(y)[:6144],validation_data=([np.array(output1)[6144:], np.array(output2)[6144:]],np.array(y)[6144:]), epochs=10, batch_size=16)

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Overfitting analysis")
axs[0].plot(list(range(1,11)), history.history['val_accuracy'], list(range(1,11)), history.history['accuracy'])

axs[0].title.set_text("Accuracy")
axs[0].legend(["validation accuracy", "training accuracy"])
axs[1].plot(list(range(1,11)), history.history['val_loss'], list(range(1,11)), history.history['loss'])
axs[1].title.set_text('Loss')
axs[1].legend(["validation loss", "trainig loss"])

## Testing

In [ ]:
transformed_test_filepaths = []
for filename in test_filepaths:
    temp = []
    for imagename in filename:
        img = cv2.imread(imagename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        face_image = img
        if(len(faces) > 0):
            xf, yf, wf, hf = faces[0]
            face_image = img[yf:yf+hf, xf:xf+wf]
        face_image = cv2.resize(face_image, (512, 512))
        

        temp.append(face_image)
    transformed_test_filepaths.append(temp)

In [ ]:
sum = 0
lis = []
frequency = {}
for i in range(0, len(transformed_test_filepaths)):
    sum += ((len(transformed_test_filepaths[i]))*(len(transformed_test_filepaths[i]) - 1))/2
    lis.append(len(transformed_test_filepaths[i]))

for item in lis:
   # checking the element in dictionary
   if item in frequency:
      # incrementing the count
      frequency[item] += 1
   else:
      # initializing the count
      frequency[item] = 1
tot_val = 0
for key in frequency:
    tot_val += key*frequency[key]
final_freq = {}
for key in frequency:
    final_freq[key] = key*frequency[key]/tot_val


# printing the frequency
print(final_freq)

### Creating pairs of images from the test dataset for evaluating the entire pipeline

In [ ]:
random.seed(42)
newX1_test = []
newX2_test = []
newY_test = []
for i in range(len(transformed_test_filepaths)):
    for j in range(0, len(transformed_test_filepaths[i])):
        for k in range(0, j):
            newX1_test.append(transformed_test_filepaths[i][k])
            newX2_test.append(transformed_test_filepaths[i][j])
            newY_test.append(0)
    
    for u in range(0, len(transformed_test_filepaths[i])):
        step = 1
        step = round((final_freq[len(transformed_test_filepaths[i])]*264)/frequency[len(transformed_test_filepaths[i])])
        no_of_iter = round(step/len(transformed_test_filepaths[i]))
        for l in range(0, 1):
            numbers = list(range(0, i)) + list(range(i+1, 49))
            r = random.choice(numbers)
            g = random.randint(0, len(transformed_test_filepaths[r]) - 1)
            newX1_test.append(transformed_test_filepaths[i][u])
            newX2_test.append(transformed_test_filepaths[r][g])
            newY_test.append(1)
for i in range(0, 79):
        
    numbers = list(range(0,i%49)) + list(range(i%49 + 1,49))
    r = random.choice(numbers)
    uu = random.randint(0,len(transformed_test_filepaths[i%49])-1)
    g = random.randint(0,len(transformed_test_filepaths[r])-1)
    newX1_test.append(transformed_test_filepaths[i%49][uu])
    newX2_test.append(transformed_test_filepaths[r][g])
    newY_test.append(1)
print(len(newY_test))

In [ ]:
c_test = list(zip(newX1_test, newX2_test, newY_test))

random.shuffle(c_test)

a_test, b_test, y_test = zip(*c_test)

In [ ]:
len(a_test)

### Extracting features using the encoder block of the AutoEncoder

In [ ]:
output1_test = []
output2_test = []
for i in range(1, len(a_test)//264 + 1):
    a1_test = a_test[(i-1)*264 : i*264]
    b1_test = b_test[(i-1)*264 : i*264]
    output1_test.extend(encoder.predict(np.array(a1_test)/255))
    output2_test.extend(encoder.predict(np.array(b1_test)/255))

In [ ]:
indexes1_test=[i for i,x in enumerate(y_test) if x == 1]
indexes0_test=[i for i,x in enumerate(y_test) if x == 0]

### Using the classifier for final training and evaluation

In [ ]:
import tensorflow.keras.backend as K
import tensorflow


def distance(vecs):
    x, y = vecs
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    
    return K.abs(x-y)


featuresA=Input(512, )
featuresB=Input(512, )
distance= Lambda(distance)([featuresA,featuresB])

x= Dense(96, activation="relu")(distance)
x= Dropout(0.3)(x)
x= Dense(64)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs=[featuresA, featuresB],outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

In [ ]:
print(len(output1))

### Fitting the classifier on the entire training data

In [ ]:
history=model.fit([np.array(output1)[:8192], np.array(output2)[:8192]],np.array(y)[:8192], epochs=15, batch_size=32)

### Obtaining final test results on the entire test data

In [ ]:
test_res = model.evaluate([np.array(output1_test)[:528], np.array(output2_test)[:528]],np.array(y_test)[:528], batch_size=16)